# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import folium
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
# import and Read the weather data and the study results

weather_metadata = "output_data/my_weather_data.csv"
weather_data = pd.read_csv(weather_metadata)
weather_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Lompoc,1,US,1581813289,72,34.64,-120.46,66.00,11.41
1,1,Nikolskoye,75,RU,1581813455,84,59.70,30.79,35.60,13.42
2,2,Kodiak,75,US,1581813456,75,57.79,-152.41,32.00,12.15
3,3,Vaini,20,TO,1581813456,66,-21.20,-175.20,84.20,10.29
4,4,Airai,0,TL,1581813456,69,-8.93,125.41,76.33,1.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
weather_data.columns
weather_data.dtypes

Unnamed: 0      int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [4]:

# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [35]:
# Store 'Lat' and 'Lng' into  locations variable 
locations = weather_data[["Lat", "Lng"]].astype(float)


hum_rate = weather_data["Humidity"].astype(float)

In [17]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)


heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
#display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather_data = weather_data.dropna()

#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weather_data.loc[weather_data["Max Temp"]<80,:]
second = first.loc[weather_data["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,4,Airai,0,TL,1581813456,69,-8.93,125.41,76.33,1.50
24,24,Cape Town,0,ZA,1581813460,93,-33.93,18.42,75.00,3.36
43,43,Hermanus,0,ZA,1581813462,74,-34.42,19.23,75.99,5.99
93,93,Bredasdorp,0,ZA,1581813468,88,-34.53,20.04,75.99,4.70
134,134,Cabedelo,0,BR,1581813475,83,-6.98,-34.83,78.80,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
4,Airai,TL,-8.93,125.41
24,Cape Town,ZA,-33.93,18.42
43,Hermanus,ZA,-34.42,19.23
93,Bredasdorp,ZA,-34.53,20.04
134,Cabedelo,BR,-6.98,-34.83


In [9]:

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 4: Airai.
Closest hotel to Airai at -8.93 , 125.41 is Kantor polisi Atsabe.
Retrieving Results for Index 24: Cape Town.
Closest hotel to Cape Town at -33.93 , 18.42 is Adderley Hotel.
Retrieving Results for Index 43: Hermanus.
Closest hotel to Hermanus at -34.42 , 19.23 is 56 Mitchell Street.
Retrieving Results for Index 93: Bredasdorp.
Closest hotel to Bredasdorp at -34.53 , 20.04 is Holiday Home Bloekomhuisie.
Retrieving Results for Index 134: Cabedelo.
Closest hotel to Cabedelo at -6.98 , -34.83 is Banda Forró Magia.
Retrieving Results for Index 239: Ouallam.
Closest hotel to Ouallam at 14.32 , 2.09 is Yansala Télécom.
Retrieving Results for Index 262: Swan Hill.
Closest hotel to Swan Hill at -35.34 , 143.55 is Jane Eliza Motor Inn.
Retrieving Results for Index 330: Gagnoa.
Closest hotel to Gagnoa at 6.13 , -5.94 is Cifpro (Centre de Formation Professionnelle de Gagnoa).
Retrieving Results for Index 484: Kontagora.
Closest hotel to Kontagora at 10.4 , 5.

In [10]:
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
4,Airai,TL,-8.93,125.41,Kantor polisi Atsabe
24,Cape Town,ZA,-33.93,18.42,Adderley Hotel
43,Hermanus,ZA,-34.42,19.23,56 Mitchell Street
93,Bredasdorp,ZA,-34.53,20.04,Holiday Home Bloekomhuisie
134,Cabedelo,BR,-6.98,-34.83,Banda Forró Magia


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map

In [60]:
weather_data.loc[weather_data['Country']=='ET']

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
139,139,Butajira,0,ET,1581813475,55,8.12,38.37,62.58,9.17
373,373,Areka,99,ET,1581813504,61,7.07,37.70,71.98,1.07


In [62]:
m = folium.Map([34.64, -120.46], zoom_start=1)
m